In [1]:
!python --version

Python 3.8.0


In [2]:
# !poetry remove pandas

In [3]:

# !poetry add tensorflow=2.5 tensorflow-text=2.5 tensorflow-hub 
# numpy=1.19.3 
# pandas==1.2.5 openpyxl

!poetry add pandas==1.2.5 openpyxl

^C


In [3]:
import tensorflow_hub as hub
import numpy as np
import tensorflow_text
import tensorflow as tf

from scipy import spatial

In [4]:
# Some texts of different lengths.
english_sentences = ["dog", "Puppies are nice.", "I enjoy taking long walks along the beach with my dog."]
italian_sentences = ["cane", "I cuccioli sono carini.", "Mi piace fare lunghe passeggiate lungo la spiaggia con il mio cane."]
japanese_sentences = ["犬", "子犬はいいです", "私は犬と一緒にビーチを散歩するのが好きです"]

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [5]:
# Compute embeddings.
en_result = embed(english_sentences)
it_result = embed(italian_sentences)
ja_result = embed(japanese_sentences)

In [6]:
en_result.numpy().shape

(3, 512)

In [7]:
# Compute similarity matrix. Higher score indicates greater similarity.
similarity_matrix_it = np.inner(en_result, it_result)
similarity_matrix_ja = np.inner(en_result, ja_result)

In [8]:
similarity_matrix_it

array([[0.9578779 , 0.3308629 , 0.30247942],
       [0.38761464, 0.73392963, 0.24814035],
       [0.2361753 , 0.21800497, 0.9283011 ]], dtype=float32)

In [9]:
import pandas as pd

In [10]:
df = pd.read_excel("ЭТ_Новости.xlsx")
df.head()

,ID,NEWS_HEAD
0,1552621575,Государственная Дума рекомендует Правительству...
1,1552115547,Евгений Грабчак: «Меры поддержки помогут ТЭК о...
2,1552115549,Павел Сниккарс: «Важная задача в электроэнерге...
3,1551704812,Вниманию гарантирующих поставщиков и энергосбы...
4,1551704814,Эдуард Шереметцев: «Национальная энергетическа...


In [11]:
%%time
df["emb"] = df["NEWS_HEAD"].apply(lambda x: embed(x).numpy()[0])

CPU times: total: 2min 13s
Wall time: 33 s


In [12]:
df.head()

,ID,NEWS_HEAD,emb
0,1552621575,Государственная Дума рекомендует Правительству...,"[6.663809e-05, 0.027403971, -0.033853978, -0.0..."
1,1552115547,Евгений Грабчак: «Меры поддержки помогут ТЭК о...,"[0.014879654, -0.001565816, 0.031823725, -0.04..."
2,1552115549,Павел Сниккарс: «Важная задача в электроэнерге...,"[-0.02432264, 0.063832596, 0.0060305446, -0.04..."
3,1551704812,Вниманию гарантирующих поставщиков и энергосбы...,"[0.029627569, 0.06477559, -0.015955614, 0.0087..."
4,1551704814,Эдуард Шереметцев: «Национальная энергетическа...,"[-0.028674185, -0.037707187, -0.0035214943, -0..."


In [13]:
df.to_pickle("../News+emb.pkl")

In [14]:
emb_ar = np.stack(df.emb.to_numpy())
emb_ar.shape

(1975, 512)

In [15]:
def get_closest_to_text(text, df, topN=5):
    print(f"Input string:\n{text}\n")
    emb_ar = np.stack(df.emb.to_numpy())
    text_emb = embed(text)
    cos_dist = np.array([spatial.distance.cosine(text_emb[0], x) for x in emb_ar])
    
    print("Closest records:")
    top_args = np.argpartition(cos_dist, topN)[:topN]
    n_df = df.copy().iloc[top_args,:]
    n_df["score"] = cos_dist[top_args]
    return n_df.sort_values("score", ascending=True)

In [16]:
get_closest_to_text(df.iloc[0].NEWS_HEAD, df, topN=5)

Input string:
Государственная Дума рекомендует Правительству РФ, несмотря на кризис, вызванный санкциями недружественных стран, сохранить рыночное ценообразование на нефтепродукты и электроэнергию на внутреннем рынке и продолжить разработку мер поддержки новых отраслей ТЭК, связанных с переходом на низкоуглеродную траекторию развития экономики.

Closest records:


,ID,NEWS_HEAD,emb,score
0,1552621575,Государственная Дума рекомендует Правительству...,"[6.663809e-05, 0.027403971, -0.033853978, -0.0...",0.000000
1742,1100853539,Комитет Государственной Думы по энергетике про...,"[-0.02631409, 0.078199565, -0.04384874, -0.012...",0.572938
1735,1100853537,Комитет Государственной Думы по энергетике про...,"[-0.018877262, 0.07468433, -0.04437423, -0.024...",0.607847
531,1255968164,Павел Завальный: «Намерение ЕС форсировать эне...,"[-0.0057070344, -0.08757431, -0.029431915, -0....",0.610514
478,1353506324,Депутаты комитета Государственной Думы по энер...,"[-0.025714295, 0.025128985, -0.03934483, -0.05...",0.614853


In [17]:
get_closest_to_text(df.iloc[2].NEWS_HEAD, df, topN=10)

Input string:
Павел Сниккарс: «Важная задача в электроэнергетике в период высокой турбулентности – сохранить отлаженные за 10 лет механизмы»

Closest records:


,ID,NEWS_HEAD,emb,score
2,1552115549,Павел Сниккарс: «Важная задача в электроэнерге...,"[-0.02432264, 0.063832596, 0.0060305446, -0.04...",0.000000
1230,1100852945,Александр Новак: «Конкурс «Лидеры энергетики» ...,"[-0.020745397, 0.020943517, 0.052267853, -0.01...",0.532184
1016,1110168307,Евгений Грабчак: «Цифровая модернизация позвол...,"[-0.028432878, 0.034383945, -0.009605763, -0.0...",0.546007
1716,1100853535,Комитет Государственной Думы по энергетике про...,"[0.014366625, 0.06528261, -0.0020122165, -0.01...",0.557654
1192,1100853113,Павел Сорокин: «РЭН-2019 соберет экспертов в н...,"[0.009449619, 0.0034493983, -0.029740725, -0.0...",0.563434
1500,1100853214,Евгений Грабчак: «Цифровые технологии позволят...,"[0.008983354, 0.06095129, 0.01233921, -0.00874...",0.565334
1194,1100853117,Павел Сниккарс провел заседание по актуализаци...,"[0.03769512, 0.041363187, 0.028168071, -0.0451...",0.566679
55,1532569895,Евгений Грабчак: «Долгосрочное планирование в ...,"[-0.031916756, 0.04177953, -0.059645936, -0.08...",0.572096
898,1157826551,Комитет Государственной Думы по энергетике про...,"[0.048303235, 0.08152408, 0.008171138, -0.0134...",0.575072
1112,1100852913,Александр Новак: «Уровень запасов топлива к зи...,"[-0.002212926, 0.05639637, 0.03731757, -0.0728...",0.579248


In [18]:
get_closest_to_text(df.iloc[18].NEWS_HEAD, df, topN=5)

Input string:
ФАС определила задачи совершенствования тарифного регулирования

Closest records:


,ID,NEWS_HEAD,emb,score
18,1546783300,ФАС определила задачи совершенствования тарифн...,"[0.09777066, 0.026426582, 0.0517174, -0.028335...",0.000000
19,1546783302,ФАС определила задачи совершенствования тарифн...,"[0.09777066, 0.026426582, 0.0517174, -0.028335...",0.000000
201,1477314954,Руководитель ФАС представил основные итоги и п...,"[0.07298895, -0.0012470628, -0.040068362, 0.01...",0.469435
728,1210095965,Цифровизация тарифного регулирования неизбежна,"[0.101398155, 0.0032326966, -0.008898718, 0.03...",0.478472
226,1470097088,"В ДОП приняты изменения, связанные с уточнение...","[0.048701253, -0.021925602, -0.025921613, 0.02...",0.494197


In [19]:
get_closest_to_text(df.iloc[19].NEWS_HEAD, df)

Input string:
ФАС определила задачи совершенствования тарифного регулирования

Closest records:


,ID,NEWS_HEAD,emb,score
18,1546783300,ФАС определила задачи совершенствования тарифн...,"[0.09777066, 0.026426582, 0.0517174, -0.028335...",0.000000
19,1546783302,ФАС определила задачи совершенствования тарифн...,"[0.09777066, 0.026426582, 0.0517174, -0.028335...",0.000000
201,1477314954,Руководитель ФАС представил основные итоги и п...,"[0.07298895, -0.0012470628, -0.040068362, 0.01...",0.469435
728,1210095965,Цифровизация тарифного регулирования неизбежна,"[0.101398155, 0.0032326966, -0.008898718, 0.03...",0.478472
226,1470097088,"В ДОП приняты изменения, связанные с уточнение...","[0.048701253, -0.021925602, -0.025921613, 0.02...",0.494197


In [20]:
get_closest_to_text(df.iloc[142].NEWS_HEAD, df)

Input string:
Уведомление для ООО "Грин Энерджи Рус"! Информация о максимально допустимых величинах потребления на собственные нужды электростанций на 2021 год!

Closest records:


,ID,NEWS_HEAD,emb,score
142,1497423434,"Уведомление для ООО ""Грин Энерджи Рус""! Информ...","[-0.0068681194, -0.0114653725, 0.038518403, -0...",0.000000
450,1353505575,"Уведомление для ООО ""Санлайт Энерджи""! Информа...","[-0.018210905, -0.030568672, 0.050093833, 0.00...",0.044107
510,1353830748,"Уведомление для ООО ""Грин Энерджи Рус"" и АО ""В...","[-0.004555098, -0.029549316, 0.03328292, -0.04...",0.050182
643,1227444065,"Уведомление для ООО ""Санлайт Энерджи""! Информа...","[-0.015239208, -0.020699883, 0.04711476, 0.001...",0.070868
666,1221731526,"Уведомление для ООО ""Санлайт Энерджи""! Информа...","[-0.015239208, -0.020699883, 0.04711476, 0.001...",0.070868


In [21]:
get_closest_to_text("I love pizza", df)

Input string:
I love pizza

Closest records:


,ID,NEWS_HEAD,emb,score
944,1138231366,С Днем энергетика!,"[0.033952687, 0.027581288, -0.021881443, 0.048...",0.817647
124,1506307579,С Днем энергетика!,"[0.033952687, 0.027581288, -0.021881443, 0.048...",0.817647
945,1138208208,С Днем Энергетика!,"[0.045147557, 0.014729201, -0.019837292, 0.042...",0.826592
1574,1100852043,В состав Ассоциации «НП Совет рынка» приняты т...,"[0.01255008, -0.0349957, -0.020117829, -0.0576...",0.826613
265,1404837295,В Члены Ассоциации «НП Совет рынка» приняты тр...,"[0.017213166, -0.02838534, -0.037241682, -0.04...",0.833423


In [28]:
import requests
import pandas as pd
import json
from pandas.io.json import json_normalize

In [30]:
index = "news-headers"
column_name = "title"
candidates = "1000"

In [31]:
def create_df_from_json(json, k):
    records = json["hits"]["hits"]
    df = pd.DataFrame()
    for i in range(0, min([len(records),int(k)])):
        df = df.append(json_normalize(records[i]), ignore_index=True)
    df.drop(["_type"], axis=1, inplace=True)
    df.rename(lambda x: x[1:], axis=1, inplace=True)
    df.sort_values("score", inplace=True, ascending=False)
    return df

In [32]:
def get_records_from_elastic(query, num=5):
    result = requests.post(f"http://127.0.0.1:80/semantic_search/", data={"index":index, "column_name":column_name, "query": query, 
                                                                       "num_results": str(candidates)}).json()
    return create_df_from_json(result, num)

In [33]:
get_records_from_elastic("I love pizza", 5)

C:\Users\roman\AppData\Local\Temp\ipykernel_7004\3538184230.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = df.append(json_normalize(records[i]), ignore_index=True)


,index,id,score,source.title
0,news-headers,1506307579,1.182353,С Днем энергетика!
1,news-headers,1138231366,1.182353,С Днем энергетика!
2,news-headers,1138208208,1.173408,С Днем Энергетика!
3,news-headers,1100852043,1.173387,В состав Ассоциации «НП Совет рынка» приняты т...
4,news-headers,1404837295,1.166577,В Члены Ассоциации «НП Совет рынка» приняты тр...


In [34]:
get_records_from_elastic(df.iloc[142].NEWS_HEAD, 5)

C:\Users\roman\AppData\Local\Temp\ipykernel_7004\3538184230.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = df.append(json_normalize(records[i]), ignore_index=True)


,index,id,score,source.title
0,news-headers,1497423434,2.000000,"Уведомление для ООО ""Грин Энерджи Рус""! Информ..."
1,news-headers,1353505575,1.955893,"Уведомление для ООО ""Санлайт Энерджи""! Информа..."
2,news-headers,1353830748,1.949818,"Уведомление для ООО ""Грин Энерджи Рус"" и АО ""В..."
3,news-headers,1227444065,1.929131,"Уведомление для ООО ""Санлайт Энерджи""! Информа..."
4,news-headers,1221731526,1.929131,"Уведомление для ООО ""Санлайт Энерджи""! Информа..."
